In [3]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV

In [4]:
# Load the time series dataset
data = pd.read_csv('../datasets/complete_data/df.csv')

# Change column names to match naming convention
data = data.rename(columns={'start MTU (UTC)': 'ds', 'Day-ahead Price [EUR/MWh] BZN|NO1': 'y'})

# Split the dataset into training and testing sets
# Split the data into training and testing sets
# X = data.drop(['y', 'ds'], axis=1)
# y = data['y']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


data.head()

,ds,y,Day-ahead Price [EUR/MWh] BZN|NO3,Day-ahead Price [EUR/MWh] BZN|NO5,Day-ahead Price [EUR/MWh] BZN|SE3,Actual Total Load [MW] - BZN|NO5,Hydro Run-of-river and poundage - BZN|NO1,Hydro Water Reservoir - BZN|NO1,Wind Onshore - BZN|NO1,Hydro Pumped Storage Aggregated- BZN|NO2,...,CBF BZN|NO2 > BZN|NO1 [MW],CBF BZN|NO1 > BZN|NO2 [MW],CBF BZN|NO3 > BZN|NO1 [MW],CBF BZN|NO1 > BZN|NO3 [MW],CBF BZN|NO5 > BZN|NO1 [MW],CBF BZN|NO1 > BZN|NO5 [MW],CBF BZN|SE3 > BZN|NO1 [MW],CBF BZN|NO1 > BZN|SE3 [MW],Stored Energy Value Water Reservoirs and Hydro Storage Plants [MWh] - BZN|NO3,Stored Energy Value Water Reservoirs and Hydro Storage Plants [MWh] - BZN|SE3
0,2022-01-01 00:00:00,129.30,41.33,129.30,41.33,1991.000000,878.0,321.0,172.0,1.0,...,1242.0,0.0,371.0,0.0,594.0,0.0,1065.0,0.0,5688093.0,1656000.0
1,2022-01-01 01:00:00,132.08,42.18,132.08,42.18,1992.666667,897.0,330.0,151.0,1.0,...,1222.0,0.0,383.0,0.0,676.0,0.0,927.0,0.0,5688093.0,1656000.0
2,2022-01-01 02:00:00,111.44,44.37,111.44,44.37,1994.333333,885.0,275.0,160.0,1.0,...,1679.0,0.0,362.0,0.0,436.0,0.0,748.0,0.0,5688093.0,1656000.0
3,2022-01-01 03:00:00,112.35,37.67,112.35,37.67,1996.000000,890.0,269.0,148.0,1.0,...,1668.0,0.0,324.0,0.0,345.0,0.0,918.0,0.0,5688093.0,1656000.0
4,2022-01-01 04:00:00,113.90,39.70,113.90,39.70,1985.500000,892.0,300.0,147.0,1.0,...,1697.0,0.0,339.0,0.0,412.0,0.0,826.0,0.0,5688093.0,1656000.0


In [11]:
def create_lagged_features(df, n_lag):
    for i in range(1, n_lag + 1):
        df[f"lag_{i}"] = df["start MTU (UTC)"].shift(i)
    return df

n_lag = 3
data = create_lagged_features(data, n_lag)
data = data.dropna()

KeyError: 'value'

In [12]:
X = data.drop("start MTU (UTC)", axis=1).values
y = data["start MTU (UTC)"].values

KeyError: "['value'] not found in axis"

In [13]:
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y.reshape(-1, 1)).ravel()

AttributeError: 'Series' object has no attribute 'reshape'

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [15]:
param_grid = {"C": [0.1, 1, 10], "gamma": ["scale", "auto", 0.1, 1]}
grid_search = GridSearchCV(SVR(kernel="rbf"), param_grid, cv=5)
grid_search.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
y_pred = grid_search.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

In [ ]:
def forecast_next_value(lagged_values, svr_model, scaler_X, scaler_y):
    scaled_lagged_values = scaler_X.transform(lagged_values.reshape(1, -1))
    forecast_scaled = svr_model.predict(scaled_lagged_values)
    forecast = scaler_y.inverse_transform(forecast_scaled.reshape(-1, 1))
    return forecast[0, 0]

# Example usage
lagged_values = np.array([100, 105, 110])  # Replace with the last n_lag values from your data
next_value_forecast = forecast_next_value(lagged_values, grid_search, scaler_X, scaler_y)
print(f"Next value forecast: {next_value_forecast}")